<a href="https://colab.research.google.com/github/snowyTheHamster/autoImageRenameCropResize/blob/master/images_auto_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Open CV Auto Crop Images

Detect the edges of an object with white background to crop images.

---

### How to use
**Prepare folders**

+ Create a project folder in Google Drive.
+ Create a folder inside project folder with images you want to crop.
+ Create another folder inside project folder to output results.

**Adjust variables and settings in step 2**

+ Make sure the names of the folders match what you created.
+ Adjust MIN_VAL and MAX_VAL if needed. (max value is 255)
+ Add extra margins to image if needed.

**Run Code**

+ Run each of the blocks in order 1 ~ 4.


### 1. Mount Google Drive

In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np

drive.mount('/content/drive/')

Mounted at /content/drive/


### 2. Adjust Settings Below

In [ ]:
# Make sure folder names match what you created in google drive
project_folder = 'images_test'
input_folder = 'images_original'
output_folder = 'images_cropped'

# define image files formats here
image_exts = [ '.jpg', '.jpeg' ]

# threshold for detecting white background, adjust if cropping is poor; otherwise prepare images with cleaner white background.
MIN_VAL = 245 # default = 210
MAX_VAL = 230 # default = 200

# for adding additional margin on the edges.
MTOP = 0 # default = 0
MRIGHT = 0 # default = 0
MBOTTOM = 0 # default = 0
MLEFT = 0 # default = 0


# dont need to change these
work_dir = os.path.join('/content/drive/My Drive/', project_folder)
dir_to_work = os.path.join(work_dir, input_folder)
dir_imgs_results = os.path.join(work_dir, output_folder)

### 3. Run Script


In [ ]:
count = 0
for _, _, files in os.walk(dir_to_work):
  file_count = len(files)
  for filename in files:
    count += 1
    file_path = os.path.join(dir_to_work, filename)
    file_name, file_ext = os.path.splitext(file_path)
    output_file_name = os.path.basename(file_name) + file_ext

    if file_ext not in image_exts:
      print("skipping " + filename + " (not a supported image file format)")
      count -= 1
      file_count -= 1
      continue

    else:
      image = cv2.imread(dir_to_work + '/' + filename)
      gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

      ret, thresh = cv2.threshold(gray, MIN_VAL, MAX_VAL, cv2.THRESH_BINARY_INV) # available options: THRESH_BINARY, THRESH_BINARY_INV, THRESH_TRUNC, THRESH_TOZERO, THRESH_TOZERO_INV
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9)) # available options: MORPH_RECT, MORPH_ELLIPSE; Also tweak last parameter(x, x)
      morphchoice = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel) # available options: MORPH_CLOSE, MORPH_OPEN, MORPH_DILATE, MORPH_ERODE

      # finding contours
      (cnts, _) = cv2.findContours(morphchoice.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      # finding the biggest coutour
      c = max(cnts, key = cv2.contourArea)

      #place c back in a list
      list_c = [ np.array( c ) ]

      # Make a rectangle from the largest coutour
      x, y, w, h = cv2.boundingRect(c)
      # Set margins for the Rectangle to prevent clipping of product
      x = x - MLEFT
      y = y - MTOP
      w = w + MLEFT + MRIGHT
      h = h + MTOP + MBOTTOM

      # draw the biggest contour (c) in white(255) with 1px border
      image = cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 1)

      # Select Images to apply changes to
      original_image = cv2.imread(dir_to_work +'/'+ filename)

      # crop the image
      cropped_image = original_image[y: y + h, x: x + w]

      # Save image to output dir
      cv2.imwrite(dir_imgs_results +'/'+ output_file_name, cropped_image)

      print(f'{count}/{file_count} Cropping {filename}... done.')


1/15 Cropping J1GA213681_1.jpg... done.
2/15 Cropping J1GA213681_4.jpg... done.
3/15 Cropping J1GA213681_5.jpg... done.
4/15 Cropping J1GA213702_3.jpg... done.
5/15 Cropping J1GA213702_4.jpg... done.
6/15 Cropping J1GA213702_5.jpg... done.
7/15 Cropping J1GA213762_3.jpg... done.
8/15 Cropping J1GA213762_4.jpg... done.
9/15 Cropping J1GA213762_5.jpg... done.
10/15 Cropping J1GA213681_3.jpg... done.
11/15 Cropping J1GA213681_2.jpg... done.
12/15 Cropping J1GA213702_1.jpg... done.
13/15 Cropping J1GA213702_2.jpg... done.
14/15 Cropping J1GA213762_1.jpg... done.
15/15 Cropping J1GA213762_2.jpg... done.


### 4. Unmount Google Drive

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
